# Pip Wheels

In [1]:
'''
!pip install pytorch_lightning
!pip install torchmetrics
!pip install tokenizers
!pip install transformers
!pip install ray[tune]
'''

'\n!pip install pytorch_lightning\n!pip install torchmetrics\n!pip install tokenizers\n!pip install transformers\n!pip install ray[tune]\n'

# Imports

In [2]:
# General Libraries
import os
import re
import random
import numpy as np
import pandas as pd
import scipy as sp



# PyTorch Lightning
import pytorch_lightning as pl
from pytorch_lightning import seed_everything, Trainer, LightningModule
from torchmetrics import Accuracy
from torchmetrics.functional import f1_score, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar 
from pytorch_lightning.loggers import TensorBoardLogger



# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix


# Ray[Tune]
import ray
from ray import air
from ray import tune
from ray.air import session
from ray.tune.integration.pytorch_lightning import TuneReportCallback

import torch
# HuggingFace Libraries
import tokenizers
import transformers 

%env TOKENIZERS_PARALLELISM=true

#out code 
from kfold_loop import KFoldLoop
from USPPM_model import USPPPM_model
from USPPM_dataset import set_tokenizer, set_max_len
from USPPM_kfold_datamodule import USPPPM_kf_datamodule

env: TOKENIZERS_PARALLELISM=true


# Configuration

## Configuration Class: notebook-specific settings

In [3]:
class CFG:
    # General
    seed = 42
    
    # Debug 
    debug = False
    debug_samples = 100

## Configuration Dictionary: trial-specific settings

In [4]:
# Defining a search space!
config_dict = {
    "target_size" : 1,
    "num_workers" : 16,
    
    # Training parameters
    "batch_size" : 32,
    "epochs" : 2,
    "n_fold" : 2,
    "warmup_steps" : 0,
    "min_lr" : 1e-6,
    "encoder_lr" : 2e-5,
    "decoder_lr" : 2e-5,
    "eps" : 1e-6,
    "betas" : (0.9, 0.999),
    "weight_decay" : 0.01,
    "fc_dropout" : 0.2,
    "seed" : 42,

    # Transformers
    # "model" : tune.choice(["microsoft/deberta-v3-large"]),
     "model" : tune.choice(["distilbert-base-uncased"]),
    #"model" : tune.grid_search(["AI-Growth-Lab/PatentSBERTa","distilbert-base-uncased","ahotrod/electra_large_discriminator_squad2_512","Yanhao/simcse-bert-for-patent","microsoft/deberta-v3-large"])
}

## Directories

In [5]:
INPUT_DIR = '../dataset/us-patent-phrase-to-phrase-matching/'
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

## Logger

In [6]:
logger = TensorBoardLogger("lightning_logs", name="USPPPM")

## Random seed

In [7]:
pl.seed_everything(CFG.seed)

Global seed set to 42


42

# Data Loading

In [8]:
cpc_texts = torch.load('cpc_texts.pth')
dataframe = pd.read_csv("dataframe.csv")
display(dataframe.head())

,Unnamed: 0,id,anchor,target,context,score,context_text,text,score_map
0,0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]abatement of pollution[SEP]HUMAN...,2
1,1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]act of abating[SEP]HUMAN NECESSI...,3
2,2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]active catalyst[SEP]HUMAN NECESS...,1
3,3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]eliminating process[SEP]HUMAN NE...,2
4,4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]forest region[SEP]HUMAN NECESSIT...,0


## Debug Slicing

In [9]:
if CFG.debug:
    dataframe = dataframe.iloc[:CFG.debug_samples,:]

## Train-Test Split

In [10]:
# train_df, test_df = train_test_split(dataframe, test_size = 0.1, random_state = CFG.seed, stratify = dataframe.score_map)
train_df, test_df = train_test_split(dataframe, test_size = 0.1, random_state = CFG.seed)
display(train_df.head())
display(test_df.head())

,Unnamed: 0,id,anchor,target,context,score,context_text,text,score_map
9900,9900,0dbb44b9a145edec,distributor pipe,pipe,B01,0.50,PERFORMING OPERATIONS; TRANSPORTING. PHYSICAL ...,distributor pipe[SEP]pipe[SEP]PERFORMING OPERA...,2
1303,1303,74afca34a5439c23,ammonia recovery,recovery of water,C01,0.25,HEMISTRY; METALLURGY. INORGANIC CHEMISTRY,ammonia recovery[SEP]recovery of water[SEP]HEM...,1
16591,16591,6371befc3ee1b0f2,inner closed,cylindrical inner member,E04,0.50,FIXED CONSTRUCTIONS. BUILDING,inner closed[SEP]cylindrical inner member[SEP]...,2
25822,25822,20489196c73bd86b,produce thin layers,produce layers,G01,0.50,PHYSICS. MEASURING; TESTING,produce thin layers[SEP]produce layers[SEP]PHY...,2
23640,23640,9af994b21c892022,parallel orientation,zero angle,G06,0.25,PHYSICS. COMPUTING; CALCULATING; COUNTING,parallel orientation[SEP]zero angle[SEP]PHYSIC...,1


,Unnamed: 0,id,anchor,target,context,score,context_text,text,score_map
33511,33511,ed1c4e525eb105fe,transmit alarm,display indicator,G08,0.00,PHYSICS. SIGNALLING,transmit alarm[SEP]display indicator[SEP]PHYSI...,0
18670,18670,5386316f318f5221,locking formation,retaining element,B60,0.25,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...,locking formation[SEP]retaining element[SEP]PE...,1
18049,18049,1544ca6753fcbddd,lateral power,transducer,H01,0.25,ELECTRICITY. BASIC ELECTRIC ELEMENTS,lateral power[SEP]transducer[SEP]ELECTRICITY. ...,1
31660,31660,f9d8979b94cec923,spreader body,spreader,A01,0.75,HUMAN NECESSITIES. GRICULTURE; FORESTRY; ANIMA...,spreader body[SEP]spreader[SEP]HUMAN NECESSITI...,3
15573,15573,e151ca5ea5cc0f08,high gradient magnetic separators,magnetic filtration,B03,0.50,PERFORMING OPERATIONS; TRANSPORTING. SEPARATIO...,high gradient magnetic separators[SEP]magnetic...,2


# Training

## Callbacks

In [11]:
ray.init(num_gpus=4)

2022-11-13 23:50:39,886	INFO worker.py:1518 -- Started a local Ray instance.


Python version:,3.8.10
Ray version:,2.0.1


In [12]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best_checkpoint",
    save_top_k=1,
    verbose=True,
    monitor='val_loss',
    mode='min'
)

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)

metrics = {"val_score": "val_score", "train_loss" : "train_loss", "val_loss" : "val_loss"}

In [13]:
def trainable(config_dict):  # Pass a "config" dictionary into your trainable.

    steps_per_epoch = len(train_df) // config_dict['batch_size']
    config_dict['training_steps'] = steps_per_epoch * config_dict['epochs']
    
    set_tokenizer(config_dict, OUTPUT_DIR)
    set_max_len(config_dict, cpc_texts, dataframe)
    # train_dataset = USPPM_dataset(config_dict)
    datamodule = USPPPM_kf_datamodule(config_dict, dataframe)
    
    model = USPPPM_model(config_dict)
    
    callbacks = [TuneReportCallback(metrics, on="validation_end"), checkpoint_callback, early_stopping_callback, TQDMProgressBar(refresh_rate=2)]
    trainer = pl.Trainer(
            logger=logger,
            num_sanity_val_steps=0,
            check_val_every_n_epoch=1,
            callbacks=callbacks,
            max_epochs=config_dict['epochs'],
            #devices=[1],
            accelerator="gpu",
        
            )
    
    internal_fit_loop = trainer.fit_loop
    trainer.fit_loop = KFoldLoop(config_dict['n_fold'], config_dict, export_path="./")
    trainer.fit_loop.connect(internal_fit_loop)
    
    trainer.fit(model, datamodule)

In [14]:
resource_group = tune.PlacementGroupFactory([{"CPU": 1, "GPU": 1}])

tuner = tune.Tuner(tune.with_resources(trainable, 
                                       {"cpu":0.25,"gpu":1}),
                                       param_space = config_dict,
                                       tune_config = tune.TuneConfig(metric="val_score", mode="max",max_concurrent_trials=4),
                                       # tune_config = tune.TuneConfig(metric="val_score", mode="max"),
                                       run_config = air.RunConfig(name="tune_uspppm", verbose=2, progress_reporter=tune.JupyterNotebookReporter(overwrite=True))
                                       )


Trial name,status,loc,model,iter,total time (s),val_score,train_loss,val_loss
trainable_9a289_00000,TERMINATED,131.114.50.210:3350669,distilbert-base_3300,4,567.619,0.613471,0.576764,0.600198


In [15]:
results = tuner.fit()

best_result = results.get_best_result()  # Get best result object
print(best_result)

2022-11-13 23:50:42,385	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
100%|██████████| 36473/36473 [00:02<00:00, 15272.36it/s]
(trainable pid=3350669) Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
(trainable pid=3350669) - This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(trainable pid=3350669) - This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a mod

(trainable pid=3350669) STARTING FOLD 1
(trainable pid=3350669) TRAIN FOLD 1 16412
(trainable pid=3350669) VALID FOLD 1 16413
Epoch 0:   0%|          | 0/1026 [00:00<?, ?it/s] 


(trainable pid=3350669) LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
(trainable pid=3350669) /storagenfs/m.petix/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
(trainable pid=3350669)   warnings.warn(
(trainable pid=3350669) 
(trainable pid=3350669)   | Name       | Type              | Params
(trainable pid=3350669) -------------------------------------------------
(trainable pid=3350669) 0 | model      | DistilBertModel   | 66.4 M
(trainable pid=3350669) 1 | criterion  | BCEWithLogitsLoss | 0     
(trainable pid=3350669) 2 | fc_dropout | Dropout           | 0     
(trainable pid=3350669) 3 | fc         | Linear            | 769   
(trainable pid=3350669) 4 | attention  | Sequential        | 394 K 
(trainable pid=3350669) -------------------------------

Epoch 0:  50%|████▉     | 512/1026 [01:39<01:39,  5.16it/s, loss=0.603, v_num=0, train_loss=0.604]
(trainable pid=3350669) 
Validation: 0it [00:00, ?it/s] 
(trainable pid=3350669) 
Epoch 0:  50%|█████     | 514/1026 [01:40<01:39,  5.13it/s, loss=0.603, v_num=0, train_loss=0.604]
(trainable pid=3350669) 
Epoch 0:  50%|█████     | 516/1026 [01:40<01:39,  5.14it/s, loss=0.603, v_num=0, train_loss=0.604]
(trainable pid=3350669) 
Epoch 0:  50%|█████     | 518/1026 [01:40<01:38,  5.15it/s, loss=0.603, v_num=0, train_loss=0.604]
(trainable pid=3350669) 
Epoch 0:  51%|█████     | 520/1026 [01:40<01:37,  5.17it/s, loss=0.603, v_num=0, train_loss=0.604]
(trainable pid=3350669) 
Epoch 0:  51%|█████     | 522/1026 [01:40<01:37,  5.18it/s, loss=0.603, v_num=0, train_loss=0.604]
(trainable pid=3350669) 
Epoch 0:  54%|█████▍    | 552/1026 [01:42<01:28,  5.39it/s, loss=0.603, v_num=0, train_loss=0.604]
(trainable pid=3350669) 
Epoch 0:  54%|█████▍    | 556/1026 [01:42<01:26,  5.41it/s, loss=0.603, v_n

(trainable pid=3350669) Epoch 0, global step 513: 'val_loss' reached 0.58302 (best 0.58302), saving model to '/storagenfs/m.petix/hlt_usppm/src/checkpoints/best_checkpoint-v24.ckpt' as top 1


...

Testing: 0it [00:00, ?it/s]0669) 
Testing DataLoader 0: 100%|██████████| 114/114 [00:06<00:00, 17.03it/s]
(trainable pid=3350669) ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
(trainable pid=3350669) ┃        Test metric        ┃       DataLoader 0        ┃
(trainable pid=3350669) ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
(trainable pid=3350669) │         test_loss         │    0.5968436002731323     │
(trainable pid=3350669) └───────────────────────────┴───────────────────────────┘
(trainable pid=3350669) TEST for FOLD 2


(trainable pid=3350669) Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
(trainable pid=3350669) - This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(trainable pid=3350669) - This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(trainable pid=3350669) Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_la

Testing: 0it [00:00, ?it/s]0669) 
Testing DataLoader 0:   0%|          | 0/114 [00:00<?, ?it/s]
(trainable pid=3350669) ensemble_avg_loss 0.56372154
(trainable pid=3350669) ensemble_avg_score 0.7995018533256664
(trainable pid=3350669) ensemble_avg_loss 0.60622925
(trainable pid=3350669) ensemble_avg_score 0.6143701551356964
Testing DataLoader 0:   2%|▏         | 2/114 [00:00<00:14,  7.52it/s]
(trainable pid=3350669) ensemble_avg_loss 0.5947206
(trainable pid=3350669) ensemble_avg_score 0.6905487889089901
(trainable pid=3350669) ensemble_avg_loss 0.62885976
(trainable pid=3350669) ensemble_avg_score 0.6951448590934086
Testing DataLoader 0:   4%|▎         | 4/114 [00:00<00:14,  7.59it/s]
(trainable pid=3350669) ensemble_avg_loss 0.57747805
(trainable pid=3350669) ensemble_avg_score 0.7470142109821758
(trainable pid=3350669) ensemble_avg_loss 0.61968285
(trainable pid=3350669) ensemble_avg_score 0.6464418795335194
Testing DataLoader 0:   5%|▌         | 6/114 [00:00<00:13,  7.82it/s]
(trai

2022-11-14 00:01:10,495	INFO tune.py:758 -- Total run time: 628.11 seconds (627.63 seconds for the tuning loop).


Result(metrics={'val_score': 0.6134706616697575, 'train_loss': 0.5767638087272644, 'val_loss': 0.6001981496810913, 'done': True, 'trial_id': '9a289_00000', 'experiment_tag': '0_model=distilbert-base-uncased'}, error=None, log_dir=PosixPath('/storagenfs/m.petix/ray_results/tune_uspppm/trainable_9a289_00000_0_model=distilbert-base-uncased_2022-11-13_23-50-43'))


In [16]:
ray.shutdown()

In [17]:
# Get a dataframe for the last reported results of all of the trials 
df = results.get_dataframe() 

In [18]:
df.to_csv('grid_search_results.csv')